In [14]:
import pandas as pd
import numpy as np
import aggregation
from utils import get_adjusted_year
import importlib
from cleaning import replace_error_values

In [15]:
# refresh aggregation
importlib.reload(aggregation)

<module 'aggregation' from 'd:\\Users\\User\\Downloads\\RA Work\\acs\\notebooks\\aggregation.py'>

In [16]:
# open elections_with_tracts.xlsx
elections_with_tracts = pd.read_excel('../input/elections_with_tracts.xlsx')
# open acs_data.csv
acs_tract_data = pd.read_csv('../output/tract_acs_data.csv')
# open statistics.xlsx
statistics = pd.read_excel('../input/statistics.xlsx')
sv = pd.read_excel('../input/selected_variables.xlsx')

In [17]:
acs_tract_data = aggregation.names_to_labels(sv, acs_tract_data)
acs_tract_data

,geocode,year,Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over,Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher,Estimate!!EMPLOYMENT STATUS!!Civilian labor force,Percent!!EMPLOYMENT STATUS!!Civilian labor force!!Unemployment Rate,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Armed Forces,Estimate!!FERTILITY!!Number of women 15 to 50 years old who had a birth in the past 12 months,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race),Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!White alone,...,Estimate!!Race alone or in combination with one or more other races!!Total population!!American Indian and Alaska Native,Estimate!!Race alone or in combination with one or more other races!!Total population!!Asian,Estimate!!Race alone or in combination with one or more other races!!Total population!!Black or African American,Estimate!!Race alone or in combination with one or more other races!!Total population!!Native Hawaiian and Other Pacific Islander,Estimate!!RACE!!Total population,Estimate!!U.S. CITIZENSHIP STATUS!!Foreign-born population,Estimate!!U.S. CITIZENSHIP STATUS!!Foreign-born population!!Not a U.S. citizen,Estimate!!HISPANIC OR LATINO AND RACE!!Total population,Estimate!!SEX AND AGE!!Total population,Estimate!!HOUSEHOLDS BY TYPE!!Total households
0,06_001_422200,2010,2301.0,1583.088,1944.0,9.5,0.0,66.0,489.0,1899.0,...,0.0,515.0,203.0,32.0,3127.0,931.0,723.0,3127.0,3127.0,1522.0
1,06_001_422400,2010,2579.0,2086.411,2715.0,1.9,0.0,0.0,276.0,1989.0,...,0.0,2362.0,90.0,16.0,4686.0,1977.0,1370.0,4686.0,4686.0,2171.0
2,06_001_422300,2010,2283.0,1566.138,2138.0,4.5,0.0,64.0,357.0,2139.0,...,0.0,782.0,173.0,44.0,3620.0,545.0,375.0,3620.0,3620.0,1591.0
3,06_001_420400,2010,2191.0,1899.597,1228.0,2.9,0.0,114.0,585.0,1035.0,...,15.0,1797.0,126.0,31.0,3472.0,2085.0,1943.0,3472.0,3472.0,1160.0
4,06_001_421800,2010,1580.0,1226.080,1194.0,6.7,0.0,4.0,317.0,1590.0,...,49.0,106.0,227.0,66.0,2162.0,218.0,99.0,2162.0,2162.0,897.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6405,50_007_002304,2023,1870.0,936.000,1717.0,2.7,0.0,23.0,49.0,2533.0,...,0.0,106.0,52.0,0.0,2814.0,396.0,171.0,2814.0,2814.0,1246.0
6406,50_007_003304,2023,4327.0,2981.000,4188.0,4.8,0.0,60.0,152.0,5289.0,...,190.0,546.0,208.0,5.0,6332.0,762.0,212.0,6332.0,6332.0,2722.0
6407,50_007_000100,2023,3429.0,1612.000,2651.0,4.6,0.0,32.0,71.0,3697.0,...,16.0,154.0,547.0,0.0,4508.0,640.0,164.0,4508.0,4508.0,1964.0
6408,50_007_000800,2023,1886.0,1402.000,1518.0,1.2,26.0,35.0,0.0,2393.0,...,0.0,10.0,19.0,0.0,2430.0,99.0,32.0,2430.0,2430.0,1032.0


In [18]:
# open elections.xlsx
elections = pd.read_excel('../input/elections.xlsx')

In [19]:
aggregated_data = []

# create df called missing_data with columns filename, year, values, num_missing, num_total, missing_tracts
missing_data = pd.DataFrame(columns=['filename', 'year', 'variable', 'values', 'num_missing', 'num_total', 'prop_missing'])

for index, row in elections_with_tracts.iterrows():
    if index % 10 == 0:
        print(f'{index} of {len(elections_with_tracts)} rows processed')

    tracts = row['tracts'].split(', ')
    year = elections[elections['filename'] == row['filename']]['year'].values[0]
    
    # get rows where geocode is in tracts and year = year and remove 0 values
    percent_dict = {tract: percent for tract, percent in zip(tracts, list(map(float, row['percents'].split(', '))))}
    tract_rows = acs_tract_data[(acs_tract_data['geocode'].isin(tracts)) & (acs_tract_data['year'] == year) & acs_tract_data['Estimate!!SEX AND AGE!!Total population'] > 0]
    percents = pd.DataFrame({'percent': [percent_dict[geocode] for geocode in tract_rows['geocode']]})
    tract_rows = pd.concat([tract_rows.reset_index(drop=True), percents], axis=1)
    tract_rows = tract_rows[tract_rows['percent'] > 0]

    election_data = {}

    for column in acs_tract_data.columns[2:]:
        # get the values in tract_rows for column
        values = tract_rows[column].values
        valid_values = values[~np.isnan(values)]

        # if all values are NaN, continue
        if all(np.isnan(values)):
            election_data[column] = np.nan
        else:
            # if at least one but not all values are NaN, add to missing data
            if any(np.isnan(values)):
                missing_data.loc[len(missing_data)] = [row['filename'], year, column, values, sum(np.isnan(values)), len(values), sum(np.isnan(values)) / len(values)]

            if column in statistics['agg_var'].values:
                pop_var = statistics[statistics['agg_var'] == column]['pop_var'].values[0]
                # if pop_var is none skip
                if pd.isnull(pop_var):
                    continue

                # if pop_var contains -
                if '-' in pop_var:
                    # split on -
                    pop_vars = pop_var.split('-')
                    pops = tract_rows[pop_vars[0]].values - tract_rows[pop_vars[1]].values
                else:
                    pops = tract_rows[pop_var].values
                
                percents = tract_rows['percent'].values
                tract_pops = pops*percents
                valid_tract_pops = tract_pops[~np.isnan(values)]

                # if sum valid_tract_pops is 0
                if sum(valid_tract_pops) == 0:
                    continue

                agg_type = statistics[statistics['agg_var'] == column]['type'].values[0]
                election_data[column] = aggregation.aggregate_statistic(agg_type, valid_values, valid_tract_pops)
            else:
                # if every value is not nan
                if any(~np.isnan(values)):
                    pops = tract_rows['Estimate!!SEX AND AGE!!Total population'].values
                    percents = tract_rows['percent'].values
                    tract_pops = pops*percents
                    # get the values in values that are not NaN as well as the corresponding values in DP05_0001E and the corresponding percents
                    valid_pops = pops[~np.isnan(values)]
                    valid_tract_pops = tract_pops[~np.isnan(values)]

                    props = valid_values / valid_pops
                    weighted_prop = aggregation.weighted_amean(props, valid_tract_pops)

                    # replace the Nan values in values with tract_pop * weighted_prop
                    for i in range(len(values)):
                        if np.isnan(values[i]):
                            values[i] = pops[i] * weighted_prop

                election_data[column] = aggregation.weighted_sum(values, percents)

    aggregated_data.append(election_data)

# save missing_data to csv
missing_data.to_csv('../output/missing_data.csv', index=False)

0 of 356 rows processed
10 of 356 rows processed
20 of 356 rows processed
30 of 356 rows processed
40 of 356 rows processed
50 of 356 rows processed
60 of 356 rows processed
70 of 356 rows processed
80 of 356 rows processed
90 of 356 rows processed
100 of 356 rows processed
110 of 356 rows processed
120 of 356 rows processed
130 of 356 rows processed
140 of 356 rows processed
150 of 356 rows processed
160 of 356 rows processed
170 of 356 rows processed
180 of 356 rows processed
190 of 356 rows processed
200 of 356 rows processed
210 of 356 rows processed
220 of 356 rows processed
230 of 356 rows processed
240 of 356 rows processed
250 of 356 rows processed
260 of 356 rows processed
270 of 356 rows processed
280 of 356 rows processed
290 of 356 rows processed
300 of 356 rows processed
310 of 356 rows processed
320 of 356 rows processed
330 of 356 rows processed
340 of 356 rows processed
350 of 356 rows processed


In [20]:
aggregated_df = pd.DataFrame(aggregated_data)
aggregated_df = pd.concat([elections_with_tracts['filename'], aggregated_df], axis=1)
aggregated_df

,filename,Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over,Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher,Estimate!!EMPLOYMENT STATUS!!Civilian labor force,Percent!!EMPLOYMENT STATUS!!Civilian labor force!!Unemployment Rate,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Armed Forces,Estimate!!FERTILITY!!Number of women 15 to 50 years old who had a birth in the past 12 months,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race),Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!White alone,Estimate!!INCOME AND BENEFITS!!Total households!!Mean household income (dollars),...,Estimate!!Race alone or in combination with one or more other races!!Total population!!American Indian and Alaska Native,Estimate!!Race alone or in combination with one or more other races!!Total population!!Asian,Estimate!!Race alone or in combination with one or more other races!!Total population!!Black or African American,Estimate!!Race alone or in combination with one or more other races!!Total population!!Native Hawaiian and Other Pacific Islander,Estimate!!RACE!!Total population,Estimate!!U.S. CITIZENSHIP STATUS!!Foreign-born population,Estimate!!U.S. CITIZENSHIP STATUS!!Foreign-born population!!Not a U.S. citizen,Estimate!!HISPANIC OR LATINO AND RACE!!Total population,Estimate!!SEX AND AGE!!Total population,Estimate!!HOUSEHOLDS BY TYPE!!Total households
0,Berkeley_11022010_CityCouncilDistrict1.csv,9022.076985,6318.375982,7650.731306,7.445267,9.976862e+00,213.011486,1559.166798,7143.477074,85367.256101,...,94.143489,2329.967285,1206.349611,97.018862,12386.623749,2548.302005,1456.299505,12386.623749,12386.623749,5773.893958
1,Berkeley_11042014_CityCouncilDistrict1.csv,10597.906584,7474.927632,8887.337828,6.116079,1.950852e+01,162.411330,1465.770059,8539.355183,94607.202016,...,250.793177,2814.230969,1454.925265,89.364090,14394.087540,2569.956015,1349.525956,14394.087540,14394.087540,6431.604448
2,Berkeley_11062018_CityCouncilDistrict1.csv,11269.226789,8154.313101,8728.608447,4.000052,0.000000e+00,120.467025,1809.850237,8326.023707,136565.780094,...,193.363980,3295.173792,1617.482305,40.962083,14922.456268,2536.341441,1318.302738,14922.456268,14922.456268,6315.832053
3,Minneapolis_11022021_CityCouncilWard1.csv,24576.637485,12252.581559,22462.264598,4.704190,2.060000e+02,519.798558,3190.014740,23468.583624,100209.535579,...,1146.263118,2100.184286,4700.094445,57.794630,34001.820047,4367.040098,2063.696526,34001.820047,34001.820047,15028.612188
4,Minneapolis_11052013_Ward13CityCouncil.csv,24033.278766,10188.472819,22309.729225,4.815993,3.720351e-09,519.880115,3483.423911,24202.113991,66995.454164,...,947.553415,2053.838466,4018.626966,106.995255,34518.066974,5084.818508,3158.060482,34518.066974,34518.066974,14353.387198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,Alaska_11082022_SenateDistrictP.csv,17179.479033,4387.018033,12856.392547,7.442783,8.352050e+02,251.192898,2150.161866,14859.692460,88133.238751,...,4801.678144,1703.732200,1996.697440,436.526331,25140.123931,1730.873970,602.840539,25140.123931,25140.123931,10605.279625
352,Alaska_11082022_SenateDistrictQ.csv,22640.571899,7174.634077,16814.578360,5.213186,2.355333e+03,569.218963,3001.978062,26317.047413,124300.851344,...,3765.968278,1847.149481,1845.363943,335.088601,36207.765620,1521.347733,820.159385,36207.765620,36207.765620,12475.472919
353,Alaska_11082022_SenateDistrictR.csv,31080.412755,10475.007075,21518.676091,5.525200,4.505171e+03,959.618788,3669.306704,31399.755220,99037.317495,...,9200.571191,1929.686078,3060.799626,498.386456,49478.863057,2583.303305,1222.776837,49478.863057,49478.863057,17962.942383
354,Alaska_11082022_SenateDistrictS.csv,17270.077985,2276.613391,13880.937268,12.422267,6.678885e+01,484.531550,1205.157182,3679.197297,87739.136515,...,22185.229240,1988.457815,629.011211,32

In [21]:
# open place_acs_data.csv
place_acs_data = pd.read_csv('../output/place_acs_data.csv')
replace_error_values(place_acs_data)
place_acs_data = aggregation.names_to_labels(sv, place_acs_data)
place_acs_data = place_acs_data.drop(columns=['year'])
# open elections.xlsx
elections = pd.read_excel('../input/elections.xlsx')
# remove geo_filename
elections = elections.drop(columns=['geo_filename'])
place_acs_data

,filename,Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over,Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher,Estimate!!EMPLOYMENT STATUS!!Civilian labor force,Percent!!EMPLOYMENT STATUS!!Civilian labor force!!Unemployment Rate,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Armed Forces,Estimate!!FERTILITY!!Number of women 15 to 50 years old who had a birth in the past 12 months,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race),Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!White alone,Estimate!!INCOME AND BENEFITS!!Total households!!Mean household income (dollars),...,Estimate!!Race alone or in combination with one or more other races!!Total population!!American Indian and Alaska Native,Estimate!!Race alone or in combination with one or more other races!!Total population!!Asian,Estimate!!Race alone or in combination with one or more other races!!Total population!!Black or African American,Estimate!!Race alone or in combination with one or more other races!!Total population!!Native Hawaiian and Other Pacific Islander,Estimate!!RACE!!Total population,Estimate!!U.S. CITIZENSHIP STATUS!!Foreign-born population,Estimate!!U.S. CITIZENSHIP STATUS!!Foreign-born population!!Not a U.S. citizen,Estimate!!HISPANIC OR LATINO AND RACE!!Total population,Estimate!!SEX AND AGE!!Total population,Estimate!!HOUSEHOLDS BY TYPE!!Total households
0,Springville_11022021_CityCouncil2yr.csv,18789.0,7419.000,16442.0,2.7,17.0,787.0,5599.0,28073.0,99894.0,...,387.0,588.0,241.0,484.0,35195.0,2441.0,1340.0,35195.0,35195.0,10258.0
1,NewYorkCity_06222021_DEMBoroughPresidentBronx.csv,943981.0,200268.000,657051.0,11.2,853.0,21559.0,814017.0,123902.0,69157.0,...,31783.0,69437.0,572513.0,4843.0,1443229.0,489783.0,231033.0,1443229.0,1443229.0,525387.0
2,NewYorkCity_06222021_DEMBoroughPresidentKings.csv,1860601.0,751724.000,1359468.0,7.3,1009.0,32190.0,506473.0,966614.0,114302.0,...,32212.0,369700.0,886411.0,5108.0,2679620.0,945466.0,358499.0,2679620.0,2679620.0,997957.0
3,NewYorkCity_06222021_DEMBoroughPresidentNewYor...,1267630.0,795444.000,953371.0,6.8,326.0,16607.0,428297.0,749800.0,184058.0,...,26798.0,228419.0,288246.0,3575.0,1645867.0,462627.0,212617.0,1645867.0,1645867.0,772601.0
4,NewYorkCity_06222021_DEMBoroughPresidentQueens...,1709759.0,594463.000,1236423.0,7.0,768.0,23609.0,661807.0,560919.0,109287.0,...,33434.0,658424.0,475771.0,7620.0,2360826.0,1112847.0,453213.0,2360826.0,2360826.0,816859.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,Cambridge_11062007_SchoolCommittee.csv,75217.0,NaN,63580.0,NaN,369.0,1330.0,7829.0,70633.0,95925.0,...,941.0,14377.0,13413.0,241.0,106255.0,27316.0,17246.0,106255.0,106255.0,47100.0
138,Cambridge_11052013_SchoolCommittee.csv,72618.0,54536.118,66288.0,5.0,68.0,1291.0,8670.0,67973.0,115608.0,...,1070.0,18428.0,13090.0,232.0,107916.0,29231.0,17696.0,107916.0,107916.0,43801.0
139,Cambridge_11072017_SchoolCommittee.csv,79240.0,62628.000,73312.0,4.0,111.0,1247.0,11066.0,69795.0,139991.0,...,903.0,22252.0,14219.0,236.0,116632.0,33701.0,21325.0,116632.0,116632.0,46835.0
140,SanFrancisco_11052019_Sheriff.csv,688969.0,410052.000,540475.0,5.3,286.0,8238.0,133241.0,339050.0,178742.0,...,12302.0,333357.0,58264.0,6942.0,865933.0,295398.0,110408.0,865933.0,865933.0,361222.0


In [22]:
# concat place_acs_data and aggregated_df
elections_data = pd.concat([place_acs_data, aggregated_df], axis=0)
# join elections_data and elections on filename
elections_data = elections_data.join(elections.set_index('filename'), on='filename')
elections_data

,filename,Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over,Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher,Estimate!!EMPLOYMENT STATUS!!Civilian labor force,Percent!!EMPLOYMENT STATUS!!Civilian labor force!!Unemployment Rate,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Armed Forces,Estimate!!FERTILITY!!Number of women 15 to 50 years old who had a birth in the past 12 months,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race),Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!White alone,Estimate!!INCOME AND BENEFITS!!Total households!!Mean household income (dollars),...,Estimate!!U.S. CITIZENSHIP STATUS!!Foreign-born population!!Not a U.S. citizen,Estimate!!HISPANIC OR LATINO AND RACE!!Total population,Estimate!!SEX AND AGE!!Total population,Estimate!!HOUSEHOLDS BY TYPE!!Total households,city,state,Geography,District #,office,year
0,Springville_11022021_CityCouncil2yr.csv,1.878900e+04,7419.000000,1.644200e+04,2.700000,17.000000,787.000000,5599.000000,28073.000000,99894.000000,...,1340.000000,3.519500e+04,3.519500e+04,10258.000000,Springville,UT,city council,NaN,City council,2021
1,NewYorkCity_06222021_DEMBoroughPresidentBronx.csv,9.439810e+05,200268.000000,6.570510e+05,11.200000,853.000000,21559.000000,814017.000000,123902.000000,69157.000000,...,231033.000000,1.443229e+06,1.443229e+06,525387.000000,New York,NY,boroughcounty,NaN,Borough president,2021
2,NewYorkCity_06222021_DEMBoroughPresidentKings.csv,1.860601e+06,751724.000000,1.359468e+06,7.300000,1009.000000,32190.000000,506473.000000,966614.000000,114302.000000,...,358499.000000,2.679620e+06,2.679620e+06,997957.000000,New York,NY,boroughcounty,NaN,Borough president,2021
3,NewYorkCity_06222021_DEMBoroughPresidentNewYor...,1.267630e+06,795444.000000,9.533710e+05,6.800000,326.000000,16607.000000,428297.000000,749800.000000,184058.000000,...,212617.000000,1.645867e+06,1.645867e+06,772601.000000,New York,NY,boroughcounty,NaN,Borough president,2021
4,NewYorkCity_06222021_DEMBoroughPresidentQueens...,1.709759e+06,594463.000000,1.236423e+06,7.000000,768.000000,23609.000000,661807.000000,560919.000000,109287.000000,...,453213.000000,2.360826e+06,2.360826e+06,816859.000000,New York,NY,boroughcounty,NaN,Borough president,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,Alaska_11082022_SenateDistrictP.csv,1.717948e+04,4387.018033,1.285639e+04,7.442783,835.205005,251.192898,2150.161866,14859.692460,88133.238751,...,602.840539,2.514012e+04,2.514012e+04,10605.279625,NaN,AK,upper state legislative,P,State senate,2022
352,Alaska_11082022_SenateDistrictQ.csv,2.264057e+04,7174.634077,1.681458e+04,5.213186,2355.333179,569.218963,3001.978062,26317.047413,124300.851344,...,820.159385,3.620777e+04,3.620777e+04,12475.472919,NaN,AK,upper state legislative,Q,State senate,2022
353,Alaska_11082022_SenateDistrictR.csv,3.108041e+04,10475.007075,2.151868e+04,5.525200,4505.170765,959.618788,3669.306704,31399.755220,99037.317495,...,1222.776837,4.947886e+04,4.947886e+04,17962.942383,NaN,AK,upper state legislative,R,State senate,2022
354,Alaska_11082022_SenateDistrictS.csv,1.727008e+04,2276.613391,1.388094e+04,12.422267,66.788850,484.531550,1205.157182,3679.197297,87739.136515,...,1302.821688,2.928110e+04,2.928110e+04,7447.485427,NaN,AK,upper state legislative,S,State senate,2022


In [23]:
# bring these columns to front: filename	city	state	Geography	District #	office	year
elections_data = elections_data[['filename', 'city', 'state', 'Geography', 'District #', 'office', 'year'] + [col for col in elections_data.columns if col not in ['filename', 'city', 'state', 'Geography', 'District #', 'office', 'year']]]
elections_data
# save as final_data.csv
elections_data.to_csv('../output/final_data.csv', index=False)